### Задание:
Необходимо разработать программу, обеспечивающую кластеризацию новостного потока с сайта [
https://www.newsvl.ru/](https://www.newsvl.ru/vlad/2019/01/19/). 
Некоторые новостные ленты содержат в себе тематические пометы. Необходимо выбрать новостную ленту с такими пометами, загрузить с нее новости, провести их кластеризацию и проверить точность кластеризации по тематическим пометам.

### Требования к заданию:
1. выгрузить новостную ленту с тематическими пометками;
2. векторизовать тексты новостей;
3. кластеризовать векторы новостей;
4. выбрать число кластеров одним из методов оценки (локтя, Дэвиса-Болдуина, …), оценить качество кластеризации по RAND Index;
5. оформить всё в виде работоспособной программы.

### Загрузка новостей
Загрузим необходимые библиотеки.

In [1]:
import requests # Загрузка новостей с сайта.
from bs4 import BeautifulSoup # Превращалка html в текст.

Выделим заголовок и текст из новостной страницы

In [79]:
def get_news_text(url):
    page_text = requests.get(url).text
    bs=BeautifulSoup(page_text, "html5lib")
    title = None
    # Заголовок
    title = bs.h1.text
    # Сам текст
    whole_text = bs.find_all('div', 'story__text')
    text = None
    for i in range(len(whole_text)):
        text = " ".join([p.text for p in whole_text[i].find_all("p")])
        if not text:
            text = " ".join(whole_text[i].text.split('\n\n'))
    return {'title': title, 
            'text': text}
    

Переберем все страницы в цикле

In [64]:
from datetime import date
from datetime import timedelta

In [60]:
def get_day_news_links(day):
    links = []
    
    # Формируем url страницы со списком новостей
    url_root = 'https://www.newsvl.ru'
    url = url_root + '/vlad/' + str(day.year) + '/' + str(day.month) + '/' + str(day.day)
    
    # получаем текст страницы
    page_text = requests.get(url).text
    bs=BeautifulSoup(page_text, "html5lib")
    
    # ветви дерева с новостями
    whole_text = bs.find_all('div', 'story-list__item')
    for i in range(len(whole_text)):
        # ссылки на новости начинаются с тега h3
        item_title = whole_text[i].find_all("h3")
        for j in range(len(item_title)):
            # Вытаскиваем ссылки в тегах a
            link = item_title[0].find('a').get('href')
            links.append(url_root + link)
    return links

In [111]:
 # Сохраняем статьи в файл.
def saveArticle(filename, art, date):
    newsfile=open(filename, 'a', encoding='utf-8')
    title = str(art['title']).encode().decode('utf-8')
    text = str(art['text']).encode().decode('utf-8')
    text = text.strip('\n')
    text = text.strip()
    newsfile.write('\n=====\n'+title)
    newsfile.write('\n' + str(date.year) + '/' + str(date.month) + '/' + str(date.day))
    newsfile.write('\n-----\n'+text)
    newsfile.close()

In [119]:
%%time
filename = r'D:\hse_iot\ml_hse_2020\data\articles_LR3.txt'
newsfile=open(filename, 'w', encoding='utf-8')
newsfile.close()
#start_date = date(2010, 1, 1)
start_date = date(2005, 1, 1)
day = start_date
one_day_delta = timedelta(days=1)
stop_date = date.today() + one_day_delta
#stop_date = date(2010, 2, 2) + one_day_delta
whole_days = stop_date - day
while day != stop_date:
    links_list = []
    links_list = get_day_news_links(day)
    for link in links_list:
        try:
            res = get_news_text(link)
            saveArticle(filename, res, day)
        except Exception as err:
            print(err)
            print('Date: ' + str(day))
            print('step: ' + str((day - start_date).days))
    print(str((day - start_date).days)+"/"+str((whole_days-one_day_delta).days), end='\r')
    
    day += one_day_delta

print(str((day - start_date).days)+"/"+str((whole_days-one_day_delta).days), end='\r')
print('Success!!!\n')

'NoneType' object has no attribute 'text'
Date: 2005-08-17
step: 228
HTTPSConnectionPool(host='www.newsvl.ru', port=443): Max retries exceeded with url: /vlad/2020/02/13/187723/ (Caused by NewConnectionError('<urllib3.connection.VerifiedHTTPSConnection object at 0x000002716269CF88>: Failed to establish a new connection: [WinError 10060] Попытка установить соединение была безуспешной, т.к. от другого компьютера за требуемое время не получен нужный отклик, или было разорвано уже установленное соединение из-за неверного отклика уже подключенного компьютера'))
Date: 2020-02-13
step: 5521
Success!!!

Wall time: 16h 18min 9s


### Обработка

In [28]:
import pandas as pd

In [123]:
newsfile = open(r'D:\hse_iot\ml_hse_2020\data\articles_LR3_last.txt', 'r', encoding='utf-8')
text_news = [(n.split("\n-----\n")[0].split('\n')[0], 
              n.split("\n-----\n")[0].split('\n')[1], 
              n.split("\n-----\n")[1]) for n in newsfile.read().split("\n=====\n")[1:]]
news = pd.DataFrame(text_news, columns = ['Header', 'Date', 'News'])
newsfile.close()

In [124]:
news.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 82282 entries, 0 to 82281
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   Header  82282 non-null  object
 1   Date    82282 non-null  object
 2   News    82282 non-null  object
dtypes: object(3)
memory usage: 1.9+ MB


In [125]:
news.tail()

,Header,Date,News
82277,ДГК отремонтировала три квартиры на Окатовой,2020/11/4,Дальневосточная генерирующая компания сообщает...
82278,117 единиц дорожной техники для Владивостока з...,2020/11/4,В этом году для муниципального предприятия «Со...
82279,Владивосток отмечает юбилей получения звания «...,2020/11/4,Ровно 10 лет Владивосток зовётся «Городом воин...
82280,В аэропорту Владивостока отменён рейс в Терней,2020/11/4,"Сегодня, 4 ноября, отменён вылет рейса HZ-534 ..."
82281,Сегодня во Владивостоке 4…6 °С,2020/11/4,"В среду, 4 ноября, во Владивостоке без осадков..."


In [122]:
date(2020, 11, 4)

datetime.date(2020, 11, 23)